In [1]:
import pinecone
import numpy as np
from nomic import atlas
import nomic
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import openai
import json
from nomic import project
from PyPDF2 import PdfReader
import pandas as pd

# for vector search
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from transformers import GPT2Tokenizer
from collections import Counter
import json
import os

In [2]:
from dotenv import load_dotenv

# load API keys from globally-availabe .env file
load_dotenv(dotenv_path="/mnt/project/chatbotai/huggingface_cache/internal_api_keys.env", override=True)

True

In [24]:
pinecone.init(api_key=os.environ['PINECONE_API_KEY_NEW_ACCT'], environment="us-east4-gcp")

index_name="uiuc-chatbot-deduped" 

In [4]:
index = pinecone.Index(index_name)

In [5]:
#tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [5]:
nomic.login('') #add own


In [6]:
#keywords
def keyword_response_API(prompt, myKwargs = {}):

  #default arguments to send the API, unless changed in function
  kwargs = {"model" :"gpt-3.5-turbo",
            "temperature" :0.6,
            "max_tokens": 500,
            "frequency_penalty":1,
            "presence_penalty":0}


  for kwarg in myKwargs:
    kwargs[kwarg] = myKwargs[kwarg]
  
  r=openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            {"role": "system", "content": "You are classifying this text into a fixed set of keywords that most closely match with the text."},
            {"role": "user", "content": prompt}
            ])
  return r['choices'][0]['message']['content']

Visualizing vectors

In [11]:
# num_vecs = 893 #900 is the limit to use query fn
# res= index.query(vector=[0.0]*1024, top_k=num_vecs, include_values=True, include_metadata=True)

In [12]:
# data = []
# for i in range(num_vecs):
#     data.append(res['matches'][i]['metadata']['text'])

In [21]:
model_name = "intfloat/e5-large"  # best text embedding model. 1024 dims.
embeddings = HuggingFaceEmbeddings(model_name=model_name)
vectorstore = Pinecone(index=index, embedding_function=embeddings.embed_query, text_key="text")

No sentence-transformers model found with name /home/nehasheth/.cache/torch/sentence_transformers/intfloat_e5-large. Creating a new one with MEAN pooling.


In [7]:
question = "What is a finite state machine in electrical engineering?"
relevant_context_list = vectorstore.similarity_search(question, k=2300)

In [8]:
len(relevant_context_list)

893

In [9]:
top_context_metadata = [f"Source: page {doc.metadata['page_number']} in {doc.metadata['textbook_name']}" for doc in relevant_context_list]
context_list = [f"{text.page_content}. {meta}" for text, meta in zip(relevant_context_list, top_context_metadata)]

In [10]:
len(context_list)

893

In [11]:
len(relevant_context_list) #with metadata, in document type format

893

Deleting atlas projects

In [ ]:
atl_proj=project.AtlasClass()
atl_proj._get_existing_project_by_name('TA Chatbot Data', 'nehas9')

In [18]:
atl_proj._delete_project_by_id('') #delete by project ID

Mapping text

Generating keywords for text. Classify based on content list. 

In [23]:
contents = [
"The Notion of Abstraction", "Hardware vs. Software", "A Computer System", "The Parts of a Computer System", "Computers as Universal Computational Devices", 
"The Algorithm", "The Program", "The ISA", "The Microarchitecture", "The Logic Circuit", 
"The Devices", "Bits, Data Types, and Operations", "The Bit as the Unit of Information", 
"Integer Data Types", "Unsigned Integers", "Signed Integers", "2's Complement Integers", "Conversion Between Binary and Decimal", 
"Binary to Decimal Conversion", "Decimal to Binary Conversion", "Extending Conversion to Numbers with Fractional Parts", 
"Operations on Bits—Part I: Arithmetic", "Addition and Subtraction", "Sign-Extension", "Overflow", 
"Operations on Bits—Part II: Logical Operations", "A Logical Variable", "The AND Function", "The OR Function", "The NOT Function", 
"The Exclusive-OR Function", "DeMorgan's Laws", "The Bit Vector", "Other Representations", 
"Floating Point Data Type (Greater Range, Less Precision)", "ASCII Codes", "Hexadecimal Notation", 
"Digital Logic Structures", "The Transistor", "Logic Gates", "The NOT Gate (Inverter)", "OR and NOR Gates", 
"Why We Can't Simply Connect P-Type to Ground", "AND and NAND Gates", "Gates with More Than Two Inputs", "Combinational Logic Circuits", 
"Decoder", "Mux", "A One-Bit Adder (a.k.a. a Full Adder)", "The Programmable Logic Array (PLA)", "Logical Completeness", 
"Basic Storage Elements", "The R-S Latch", "The Gated D Latch", "The Concept of Memory", "Address Space", "Addressability",
"A 22-by-3-Bit Memory", "Sequential Logic Circuits", "A Simple Example: The Combination Lock", "The Concept of State", 
"The Finite State Machine and Its State Diagram", "The Synchronous Finite State Machine", "The Clock", "Preview of Coming Attractions: The Data Path of the LC-3", 
"The von Neumann Model", "Basic Components", "Memory", "Processing Unit", "Input and Output", "Control Unit", "The LC-3: An Example von Neumann Machine", 
"Instruction Processing", "The Instruction", "The Instruction Cycle (NOT the Clock Cycle!)", "Changing the Sequence of Execution", 
"Control of the Instruction Cycle", 
"Halting the Computer (the TRAP Instruction)", 
"Our First Program: A Multiplication Algorithm", 
"Exercises", 
"The LC-3", 
"The ISA: Overview", 
"Memory Organization", 
"Registers", 
"The Instruction Set", 
"Opcodes", 
"Data Types", 
"Addressing Modes", 
"Condition Codes", 
"Operate Instructions", 
"ADD, AND, and NOT", 
"Immediates", 
"The LEA Instruction (Although Not Really an Operate)", 
"Data Movement Instructions", 
"PC-Relative Mode", 
"Indirect Mode", 
"Base+offset Mode", 
"An Example", 
"Control Instructions", 
"Conditional Branches", 
"Two Methods of Loop Control", 
"The JMP Instruction", 
"The TRAP Instruction", 
"Another Example: Counting Occurrences of a Character", 
"The Data Path Revisited", 
"Basic Components of the Data Path", 
"The Instruction Cycle Specific to the LC-3", 
"Exercises", 
"Problem Solving", 
"Systematic Decomposition", 
"The Three Constructs: Sequential, Conditional, Iterative", 
"LC-3 Control Instructions to Implement the Three Constructs", 
"The Character Count Example from Chapter 5, Revisited", 
"Debugging", 
"Debugging Operations", 
"Use of an Interactive Debugger", 
"Exercises", 
"Assembly Language Programming—Moving Up a Level", 
"An Assembly Language Program", 
"Instructions", 
"Pseudo-Ops (AssemblerDirectives)", 
"Example: The Character Count Example of Section 5.5, Revisited Again!", 
"The Assembly Process", 
"Introduction", 
"A Two-Pass Process", 
"The First Pass: Creating the Symbol Table", 
"The Second Pass: Generating the Machine Language Program",
"Beyond the Assembly of a Single",
"The Executable Image", "More than One Object File", "A The LC-3 ISA","A.2 The Instruction Set", "A.3 Interrupt and Exception Processing", 
"Interrupts", "Exceptions", "From LC-3 to x86", "LC-3 Features and Corresponding x86 Features", "Instruction Set", 
"Memory", "Internal State", "The Format and Specification of x86 Instructions 690", "Prefix 691", 
"Opcode", "ModR/M Byte", "SIB Byte",
"Displacement", "Immediate", "An Example", 
"The Microarchitecture of the LC-3", "Overview of LC-3", "The State Machine", 
"The Data Path", "The Control Structure", "The TRAP Instruction", 
"Memory-Mapped I/O", "Interrupt and Exception Control", "Initiating an Interrupt", 
"Returning from an Interrupt or Trap Service Routine, RTI", "Initiating an Exception", 
" Control Store", "The C Programming Language", "Overview", "C Conventions", 
"Source Files", "Header Files", "Comments", "Literals", "Formatting", 
"Keywords", "Types", "Basic Data Types", "Type Qualifiers", "Storage Class", 
"Derived Types", "typedef", "Declarations", "Variable Declarations", 
"Function Declarations", "Operators", "Assignment Operators", "Arithmetic Operators", 
"Bit-Wise Operators", "Logical Operators", "Relational Operators", 
"Increment/Decrement Operators", "Conditional Expression Operators", 
"Pointer, Array, and Structure Operators", "sizeof", "Order of Evaluation"]


In [24]:
len(contents)

189

In [12]:
len(data)

900

In [25]:
len(relevant_context_list)

2301

In [ ]:
#checking number of times we are exceeding the token limit
counter = 0
for i, chap in enumerate(relevant_context_list):
    number_of_tokens = len(tokenizer(str(chap))['input_ids'])
    print(number_of_tokens)
    if number_of_tokens>4096:
        print("to remove %s" % i )

In [ ]:
text_labelled = []
for i, dat in enumerate(relevant_context_list):
    print(i)
    #print(dat)
    text = {}
    text['text'] = dat
    prompt = "Assign exactly one keyword from the below list of keywords to this text. Make sure it captures the content of the text perfectly. If no keyword matches, assign the most relevant next keyword. DO NOT classify any of them as None or N/A. Every text must have a keyword. Only output the keyword assigned without any other extra words. Keywords: %s Text : %s" % (str(contents), dat) 
    keywords = keyword_response_API(prompt)
    text['labels'] = keywords
    text_labelled.append(text)
    print("done")

In [20]:
len(text_labelled)

1663

In [ ]:
keywords = [keyword for keyword in keywords if not keyword.startswith('None of')]

In [22]:
df = pd.DataFrame(text_labelled)
df.head() #this has alot of n/A so we cannot use this data to map!!

,text,labels
0,"page_content=""So what if I'm in the driver sta...",Finite State Machine
1,page_content='82 3.1.3 Finite State Machines ...,Finite State Machines
2,"page_content=""I push a button a couple of time...",Finite State Machine
3,"page_content=""So go build a 10 operand compara...",Combinational Logic Circuits
4,"page_content=""So what if I'm in the driver sta...",Finite State Machine


In [24]:
c = df['labels'].value_counts()

In [26]:
df['labels'] =  df['labels'].apply(lambda x: '' if x.startswith(('N/A', 'None')) else x)


In [27]:
df = df.replace('', pd.NaT)

In [28]:
df.isnull().sum()

text        0
labels    146
dtype: int64

In [29]:
df.dropna(inplace=True)

In [30]:
df.labels.value_counts()

Hardware vs. Software                           164
The Notion of Abstraction                       159
Digital Logic Structures                        130
Sequential Logic Circuits                        98
Finite State Machine                             75
                                               ... 
Serialization and Finite State Machines           1
Opcodes                                           1
The Assembly Process                              1
The LC-3: Interrupt and Exception Processing      1
Bayes rule                                        1
Name: labels, Length: 180, dtype: int64

In [71]:
df.to_pickle("df.pkl")

Visualize sample

In [26]:
df = pd.read_pickle("/home/nehasheth/chatbotai/neha/data-generator/input_data/data_exploration/df.pkl")
df.shape

(1517, 2)

In [27]:
df.head(100)

,text,labels
0,"page_content=""So what if I'm in the driver sta...",Finite State Machine
1,page_content='82 3.1.3 Finite State Machines ...,Finite State Machines
2,"page_content=""I push a button a couple of time...",Finite State Machine
3,"page_content=""So go build a 10 operand compara...",Combinational Logic Circuits
4,"page_content=""So what if I'm in the driver sta...",Finite State Machine
...,...,...
95,page_content='3.2 Finite State Machine Design ...,Finite State Machine Design Examples
96,"page_content='First, the two external inputs: ...",Sequential Logic Circuits
97,"page_content=""The entire signal is shifted to ...",The Notion of Abstraction
98,"page_content=""As you'll see, it'll be one to f...",The von Neumann Model


In [34]:
duplicate = df[df.duplicated('text')]

In [31]:
df.drop_duplicates(subset='text', keep='first', inplace=True)

In [32]:
df.labels.nunique()

137

In [33]:
df.shape

(491, 2)

In [36]:
df.head(20)

,text,labels
0,"page_content=""So what if I'm in the driver sta...",Finite State Machine
1,page_content='82 3.1.3 Finite State Machines ...,Finite State Machines
2,"page_content=""I push a button a couple of time...",Finite State Machine
3,"page_content=""So go build a 10 operand compara...",Combinational Logic Circuits
5,"page_content=""I'll just flip through it, and t...",Finite State Machine
8,page_content='3.1 Serialization and Finite Sta...,Finite State Machines
9,"page_content=""Every state that has a single ar...",The von Neumann Model
10,"page_content=""But you'll see, if you look unde...",Finite State Machines
11,"page_content=""Likewise, if I was in open angry...",State Machines
12,"page_content=""And I'll try to remind you later...",Digital Logic Structures


In [10]:
df['text'] = df['text'].astype(str)
df['labels'] = df['labels'].astype(str)

In [11]:
df.dtypes

text      object
labels    object
dtype: object

In [12]:
documents = df.to_dict('records')

In [14]:
for i, t in enumerate(documents):
    metadata_index = (documents[i]['text']).index('lookup_str')
    documents[i]['text'] = documents[i]['text'][:metadata_index]
    

In [15]:
len(documents)

1517

In [19]:
project = atlas.map_text(data=documents,
                          indexed_field='text',
                          colorable_fields=['labels'],
                          name = 'TA Chatbot Data',
                          build_topic_model = True,
                          is_public=True
                          )

2023-03-27 01:02:27.833 | INFO     | nomic.project:_create_project:942 - Creating project `TA Chatbot Data` in organization `nehas9`
2023-03-27 01:02:28.998 | INFO     | nomic.atlas:map_text:210 - Uploading text to Atlas.
2it [00:01,  1.20it/s]                       
2023-03-27 01:02:30.672 | INFO     | nomic.atlas:map_text:227 - Text upload succeeded.
2023-03-27 01:02:31.764 | INFO     | nomic.project:create_index:1252 - Created map `TA Chatbot Data` in project `TA Chatbot Data`: https://atlas.nomic.ai/map/d74a8110-37ac-41be-904e-0e6b39bfa1f4/cc7a0bca-97c7-4764-90e5-c5b06bd01d7f
2023-03-27 01:02:31.765 | INFO     | nomic.atlas:map_text:241 - TA Chatbot Data: https://atlas.nomic.ai/map/d74a8110-37ac-41be-904e-0e6b39bfa1f4/cc7a0bca-97c7-4764-90e5-c5b06bd01d7f
